Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 657 µs, sys: 3.97 ms, total: 4.63 ms
Wall time: 10.2 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.255.13/8341/1 Dashboard: http://10.20.255.13:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 287 ms, sys: 45 ms, total: 332 ms
Wall time: 2.51 s


[1.6286041937471265,
 1.6668582134467644,
 1.0987198295673735,
 1.3025401327283723,
 1.0391991965008218,
 1.9578228056358071,
 1.3308025699015953,
 1.844582143015287,
 1.365642380031004,
 1.5560770865333837,
 1.9552625792590952,
 1.9584115452177198,
 1.5087767813166657,
 1.4764895978516805,
 1.1436908540244068,
 1.0745396717029054,
 1.75085656427805,
 1.766930583618601,
 1.6300257563278957,
 1.533979215649116,
 1.1208043478470406,
 1.7304708514016953,
 1.24752831320758,
 1.1400447514196725,
 1.6510737399642763,
 1.9482181573429223,
 1.4013051175051694,
 1.8190621546259151,
 1.9349411687997715,
 1.3130275057496508,
 1.7788370620992238,
 1.4834034919176298,
 1.1707790845098471,
 1.341316850826162,
 1.2979950582584054,
 1.475181061903637,
 1.3509376004304199,
 1.8234898378230096,
 1.0256651014086486,
 1.5852890596932179,
 1.0887329529813936,
 1.6333873816768656,
 1.886987753725629,
 1.3574735585617197,
 1.4023819285722747,
 1.2465593739428589,
 1.7566062411213388,
 1.1766058220353397,
 1.

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.8401813096369732,
 1.4951067255114032,
 1.7168526520950462,
 1.420430742687571,
 1.1483000137753963,
 1.9544161657090071,
 1.16255549718553,
 1.6203949238601543,
 1.6828210310075784,
 1.4504573241695047,
 1.9037897279315508,
 1.8754642794527556,
 1.8458684087661976,
 1.653407040911301,
 1.6067605575198507,
 1.205863562987298,
 1.9828747902384083,
 1.1650949269724582,
 1.6039505606680735,
 1.36410938425852,
 1.2882422260547006,
 1.8004522357017672,
 1.0420794261770268,
 1.614077766194281,
 1.6926492259005133,
 1.259636654409909,
 1.6777758874374058,
 1.9628286310245535,
 1.1946679062099022,
 1.0505049857942392,
 1.7663402647494104,
 1.4885600525936116,
 1.8642022653944084,
 1.223633365590958,
 1.9025970083713568,
 1.917619007312168,
 1.6891883902657094,
 1.2119664021980103,
 1.2545728139748629,
 1.9538440247263549,
 1.722279411472793,
 1.8919317180119641,
 1.2717453183688916,
 1.5244895928227677,
 1.249621171896835,
 1.4357228090654441,
 1.5665888055127417,
 1.0638112937797524,
 1.35

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 249 ms, sys: 34.8 ms, total: 284 ms
Wall time: 2.51 s
